# Exercise: Flood Analysis in Opole (2024) using Sentinel-1 SAR Data

In this exercise, you will compare radar backscatter from Sentinel-1 before and after the 2024 flood in Opole, Poland.

**Objective:**
- Load Sentinel-1 VV data before and after the flood
- Convert radar intensity to decibels (dB)
- Calculate the difference between the two dates
- Identify areas with a significant decrease in VV (possible flood zones)

## Step 1: Import Earth Engine and geemap

In [1]:
import ee
import geemap
#ee.Authenticate()
ee.Initialize(project='ee-kkosciukk')

In [2]:
Map = geemap.Map(center=[50.675, 17.931], zoom=12)
region = ee.Geometry.Point([17.931, 50.675])

## Step 2: Load Sentinel-1 VV images (before and after flood)

In [22]:
# TODO: Load Sentinel-1 collection for before and after flood
# Use filterBounds, filterDate, and select 'VV' band
# Apply .mean() to get average image from each period


before_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(region) \
    .filterDate('2024-08-01', '2024-08-30') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV') \
    .mean()

after_flood = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(region) \
    .filterDate('2024-09-10', '2024-09-15') \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV') \
    .mean()


## Step 3: Convert to dB scale

In [23]:
# TODO: Use .log10().multiply(10) to convert before/after images to dB

before_flood_db = before_flood.log10().multiply(10)
after_flood_db = after_flood.log10().multiply(10)


## Step 4: Calculate difference and threshold potential flood areas

In [24]:
# TODO: Subtract after_dB from before_dB
# Create flood mask: pixels where difference > 3 dB

diff_db = before_flood_db.subtract(after_flood_db)


flood_mask = diff_db.gt(3)


## Step 5: Visualize the results

In [25]:
# TODO: Use Map.addLayer() to show before, after, difference, and flood mask
# Visualization parameters
vv_params = {'min': -25, 'max': 0}
diff_params = {'min': -5, 'max': 5}
mask_params = {'palette': ['white', 'blue']}


mask_vis = {'palette': ['000000', 'ff0000'], 'opacity': 1.0}
Map.addLayer(flood_mask.updateMask(flood_mask), mask_vis, 'Flood Mask (Enhanced)')
Map.addLayer(before_flood_db, vv_params, 'Before Flood (dB)')
Map.addLayer(after_flood_db, vv_params, 'After Flood (dB)')
Map.addLayer(diff_db, diff_params, 'Difference (dB)')
Map.addLayer(flood_mask.updateMask(flood_mask), mask_params, 'Flood Mask')

Map


Map(bottom=2820670.0, center=[50.66518583231177, 17.92749881744385], controls=(WidgetControl(options=['positio…